In [1]:
import pandas as pd

In [2]:
from openai import OpenAI

client = OpenAI()

In [56]:
df=pd.read_csv('../data/clean_data/Final_QandA_split_conversations_with_keywords.csv')

In [57]:
df.head()

,Chunked_Content,number of sentences,number of words,Keywords
0,when you eat it alone?. Good idea to eat it on...,2,69,"['pass', 'meal', 'home', 'nausea', 'honey', 'w..."
1,So they ate clay?. Act Charcoal will do the jo...,5,34,"['pass', 'meal', 'job', 'cold', 'neutral', 'cl..."
2,So when you have enough fat you feel nausea if...,1,13,"['eat', 'nausea', 'fat', 'feel']"
3,"So guys, water=bad?. I have dry skin even if i...",2,14,"['twice', 'water', '=', 'drink', 'skin', 'guy'..."
4,@ubeats1 could you please advice any source/bo...,8,110,"['detoxification', 'meal', 'trouble', 'literat..."


In [58]:
df.reset_index(level=0, inplace=True) 
df.rename(columns={'index':'id'}, inplace=True)

In [59]:
documents = df.to_dict(orient='records')
documents[0:2]

[{'id': 0,
  'Chunked_Content': "when you eat it alone?. Good idea to eat it once wuth honey and once alone My kids eat it with the meal.They aren't at home btw meals I will do it during week end Yesterday I had nausea after several hours passed from second meal.Today I had zero hunger until 20h+ passed after previous that meal Today I made only one typical meal enhanced with 4 egg yolks",
  'number of sentences': 2,
  'number of words': 69,
  'Keywords': "['pass', 'meal', 'home', 'nausea', 'honey', 'week', 'hunger', 'second', 'previous', 'btw', '20h+', 'kid', 'good', 'typical', 'idea', 'enhance', 'zero', '4', 'today', 'eat', 'yolk', 'egg', 'yesterday', 'wuth', 'hour', 'end']"},
 {'id': 1,
  'Chunked_Content': 'So they ate clay?. Act Charcoal will do the job?. 4,5 h passed I feel neutral, zero hunger and body warmed itself to confort temperature.. Before meal I felt colder than normal.. Thx Ring',
  'number of sentences': 5,
  'number of words': 34,
  'Keywords': "['pass', 'meal', 'job

In [61]:
prompt_template = """
You emulate a user of our health adviser application.
Formulate 5 questions this user might ask based on chunked content of conversation.
Make the questions specific to that chunk.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

Chunked_Content: {Chunked_Content}


Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [62]:
prompt = prompt_template.format(**documents[0])

In [63]:
prompt

'You emulate a user of our health adviser application.\nFormulate 5 questions this user might ask based on chunked content of conversation.\nMake the questions specific to that chunk.\nThe record should contain the answer to the questions, and the questions should\nbe complete and not too short. Use as fewer words as possible from the record. \n\nThe record:\n\nChunked_Content: when you eat it alone?. Good idea to eat it once wuth honey and once alone My kids eat it with the meal.They aren\'t at home btw meals I will do it during week end Yesterday I had nausea after several hours passed from second meal.Today I had zero hunger until 20h+ passed after previous that meal Today I made only one typical meal enhanced with 4 egg yolks\n\n\nProvide the output in parsable JSON without using code blocks:\n\n{"questions": ["question1", "question2", ..., "question5"]}'

In [64]:
print(_)

You emulate a user of our health adviser application.
Formulate 5 questions this user might ask based on chunked content of conversation.
Make the questions specific to that chunk.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

Chunked_Content: when you eat it alone?. Good idea to eat it once wuth honey and once alone My kids eat it with the meal.They aren't at home btw meals I will do it during week end Yesterday I had nausea after several hours passed from second meal.Today I had zero hunger until 20h+ passed after previous that meal Today I made only one typical meal enhanced with 4 egg yolks


Provide the output in parsable JSON without using code blocks:

{"questions": ["question1", "question2", ..., "question5"]}


In [65]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [73]:
questions = llm(prompt)
questions

'{"questions": ["Is it advisable to eat this food alone or with honey?", "How do your kids typically consume this food during meals?", "When are you planning to try this food again since your kids aren\'t home?", "What symptoms did you experience after your second meal yesterday?", "What did you add to your typical meal today?"]}'

In [67]:
import json

In [68]:
json.loads(questions)

{'questions': ['Is it a good idea to eat honey with the food once and eat it alone another time?',
  'Do your kids eat the food with their meals regularly?',
  'Are your kids typically at home during meals or away?',
  'What caused your nausea after your second meal yesterday?',
  'How many egg yolks did you enhance your typical meal with today?']}

In [69]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [70]:
from tqdm.auto import tqdm

In [71]:
results = {}

In [72]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions_raw = generate_questions(doc)
    questions = json.loads(questions_raw)
    results[doc_id] = questions['questions']

  0%|          | 0/49006 [00:00<?, ?it/s]

KeyboardInterrupt: 